In [1]:
# General imports
import numpy as np

# Pre-defined ansatz circuit and operator class for Hamiltonian
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp

# SciPy minimizer routine
from scipy.optimize import minimize

# Plotting functions
import matplotlib.pyplot as plt

In [2]:
from qiskit import *
from qiskit.visualization import *
import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session
from qiskit.primitives import Sampler
# Loading IBM Quantum account
service = QiskitRuntimeService(channel="ibm_quantum", token="8558f42abd5ce4c37e51ce37375293303fab09a45f771700ee214028841a7a19c5a8eedca85b84770d7d51177d9869776d88e8adb34bbf0a91b6c607a2e5d1a7")
backend = service.least_busy(operational=True, simulator=False)

In [3]:
def test_ansatz(params):
    ansatz = EfficientSU2(size)
    ansatz.measure_all()

    pub = (ansatz, params)
    job = sampler.run([pub], shots=shots)
    
    result = job.result()[0]

    vector = dict_to_vec(result.data.meas.get_int_counts(), size)
    for i in range(len(vector)):
        vector[i] /= shots
    
    return vector

In [4]:
def cost_func(params):
    ansatz = EfficientSU2(size)
    ansatz.measure_all()

    pub = (ansatz, params)
    job = sampler.run([pub], shots=shots)
    
    result = job.result()[0]

    vector = dict_to_vec(result.data.meas.get_int_counts(), size)

    value = 0
    for i in range(len(vector)):
        value += np.abs((vector[i]/shots)-target_vector[i])
    
    return value

In [5]:
def dict_to_vec(dict, qubit_size):
    keys_list = list(dict.keys())

    vector = []
    for i in range(2**qubit_size):
        vector.append(dict.get(i, 0))
        
    return vector

In [6]:
from qiskit import QuantumCircuit
from qiskit.primitives import StatevectorSampler

from memory_profiler import memory_usage

with Session(backend=backend) as session:
    size = 4
    shots = 1000
    target_vector = np.random.random(2**size)

    norm = 0
    for i in target_vector:
        norm += i*i
    for i in range(len(target_vector)):
        target_vector[i] /= norm
    
    sampler = StatevectorSampler()
    
    error = 1000000000000000000
    while error > 100000000:
        x0 = 2 * np.pi * np.random.random(size*8)
        result = minimize(cost_func, x0, method='COBYLA', options={'tol': 1e-4, 'disp': True})
        error = result.fun
        print(error)

    print("Optimal parameters:", result.x)
    print("Minimum function value:", result.fun)

    v = test_ansatz(result.x)

    d = []
    for i in range(len(v)):
        d.append(v[i]-target_vector[i])
    print(d)

0.7257255436748351
Optimal parameters: [4.15450596 3.77640529 3.75555552 2.20204902 4.7006059  4.73341733
 2.49149954 1.31861531 0.66373789 1.87506943 4.6672516  1.50874672
 2.88175639 3.04497431 0.39820091 2.27317946 2.17241356 6.19692066
 6.2550553  6.52951097 1.2020304  0.7687695  3.84608055 1.60942254
 4.63794776 4.93971671 6.65139438 4.61352641 4.32083613 3.89005844
 2.28719502 2.68537083]
Minimum function value: 0.7257255436748351
[-0.05724101081395527, -0.023649777950668083, -0.02392368508563868, -0.08225198369717561, -0.003568116285293869, 0.008759582405203352, -0.015332894802475378, -0.10435484233930611, -0.014414495308430517, -0.0786583105827807, -0.04386019963949786, -0.010163580737443702, -0.12998749081718042, -0.0386066082154697, -0.026502697396154684, -0.06245026759816114]


In [16]:
from memory_profiler import memory_usage

def ability_test():
    error = 1000000000000000000
    while error > 100000000:
        x0 = 2 * np.pi * np.random.random(size*8)
        result = minimize(cost_func, x0, method='COBYLA', options={'tol': 1e-4, 'disp': True})
        error = result.fun
    
    v = test_ansatz(result.x)
    return v

with Session(backend=backend) as session:
    size = 5
    shots = 1000
    target_vector = np.random.random(2**size)

    norm = 0
    for i in target_vector:
        norm += i*i
    for i in range(len(target_vector)):
        target_vector[i] /= norm
    
    sampler = StatevectorSampler()
    
    mem_usage = memory_usage(ability_test, interval=1)
    print("Average memory usage during optimization (MB):", sum(mem_usage) / len(mem_usage))

Average memory usage during optimization (MB): 247.06417410714286


In [22]:
import time

with Session(backend=backend) as session:
    size = 5
    shots = 1000
    target_vector = np.random.random(2**size)

    norm = 0
    for i in target_vector:
        norm += i*i
    for i in range(len(target_vector)):
        target_vector[i] /= norm
    
    sampler = StatevectorSampler()
    
    start_time = time.time()
    ability_test()
    end_time = time.time()
    print("Time elapsed during optimization (s):", end_time - start_time)

Time elapsed during optimization (s): 3.901832342147827
